<a href="https://colab.research.google.com/github/ulissesaalencar/ET_SDTF/blob/main/STEEP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Evapotranspiration in the FTSS <br> <br> Research Group: Ulisses Bezerra, John Cunha, Fernanda Valente, Rodolfo Nóbrega, João Andrade, Magna Moura, Anne Verhoef, Aldrin Marin, Carlos Galvão. <br> <br> UNIVERSITY - UNIVERSIDADE FEDERAL DE CAMPINA GRANDE - UFCG <br> <br> LAB - Laboratory of Hydraulic-BU <br> <br> BRAZIL, CAMPINA GRANDE - PB <br> <br> Script developed by: Ulisses Bezerra em: (08.06.2021), Revised by: John Cunha, João Andrade. <br> <br> Last change 02.06.2022<br> <br> Contact: ulisses.alencar@estudante.ufcg.edu.br
---

# STEEP - Seasonal Tropical Ecosystem Energy Partitioning

## Initial requirements

### Loading packages

In [ ]:
# !pip install geemap
import os
import ee
import geemap
import hydrostats.metrics as hm
import numpy as np
import pandas as pd
import datetime
import pprint
import math
import matplotlib.pyplot as plt

Map = geemap.Map()

### Date start/end

In [ ]:
#set the year!
year = 2014

start = str(year) + '-' + '01' + '-01'
end = str(year+(1)) + '-' + '01' + '-01'

dates = pd.date_range(start=pd.datetime(year, 1, 1),end=pd.datetime(year, 12, 31))

<ipython-input-2-4337b71438f0>:7: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  dates = pd.date_range(start=pd.datetime(year, 1, 1),end=pd.datetime(year, 12, 31))


### List of coefficients

In [ ]:
#List of coefficients
# Kelvin to Celsius
c3 = ee.Number(273.15)
# Von karman constant
von = ee.Number(0.41)
# Stefan-Boltzmann constant
sigma_SB = ee.Number(0.000000056704)

### ROI

In [ ]:
#set the coordinate!
pts = ee.Geometry.Point([-37.2514, -6.5783]) 
pts1 = pts.buffer(500000);

caatinga = ee.FeatureCollection('users/ulissesalencar17/caatinga')
pts1 = pts1.bounds()

ROI = pts1
roi = ROI

def clipper(image):
    return image.clip(caatinga)

## Input data

### Weather inputs 

#### ERA5-land

In [ ]:
#UTC in Caatinga BR is -3 
climatic_terra = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY") \
.filter(ee.Filter.date(start, end)) \
.filter(ee.Filter.calendarRange(14,14,'hour')) \
.map(clipper)

climatic_day = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY") \
.filter(ee.Filter.date(start, end)) \
.select(['surface_net_solar_radiation_hourly','surface_net_thermal_radiation_hourly']) \
.map(clipper)
                
inidate = ee.Date(start) 
enddate = ee.Date(end) 

difdate = enddate.difference(inidate, 'day')

# Time lapse
lapse = ee.List.sequence(0, difdate.subtract(1))

def func_pts(day):
    return inidate.advance(day, 'day')

listdates = lapse.map(func_pts)

# Daily dataset
def func_cac_mean(day):
    day = ee.Date(day)
    daily_collection = climatic_day.filter(ee.Filter.date(day, day.advance(1, 'day')))
    convert_Rs = daily_collection.select(['surface_net_solar_radiation_hourly']);
    convert_Rl = daily_collection.select(['surface_net_thermal_radiation_hourly']);
    convert_Rs_day = convert_Rs.sum().divide(86400);
    convert_Rl_day = convert_Rl.sum().divide(86400);
    Rn_d = (convert_Rs_day.add(convert_Rl_day)).rename('Rn_d');
    return Rn_d.set('system:index', day.format('YYYY-MM-dd')) \
    .set('date', day.format('YYYY-MM-dd')) .set('system:time_start', day.millis()) \
    .set('system:time_end', day.millis())

#RN24h
Rn24 = ee.ImageCollection.fromImages(listdates.map(func_cac_mean))
Rn24 = (Rn24.select(['Rn_d'])).toBands()

#### Constant image

In [ ]:
def img_const(image):
    img_1 = ee.Image.constant(1).toFloat().rename('img_1')
    return image.set('system:time_start',image.get('system:time_start')) \
    .addBands(ee.Image(img_1)) \
    .clip(roi)

img_1 = (climatic_terra.map(img_const)).select(['img_1'])
img_1 = img_1.toBands()

#### Inputs weather
* Air temperature (K) (2m);  
* dewpoint temperature (2m);  
* u/v component of wind (10m);

In [ ]:
#Air temperature 
temperature_terra = climatic_terra.select(['temperature_2m'])
Tbands_terra = temperature_terra.toBands()
T_air_band_K_terra = Tbands_terra
T_air_band_C_terra = Tbands_terra.subtract(c3)

#SRTM
srtm = ee.Image('USGS/SRTMGL1_003')

elevation = srtm.select('elevation')
#elevation -> 365 bands
elevation_1 = img_1.multiply(elevation)
alt_mean_img = elevation_1.clip(ROI)

img_const_1 = (alt_mean_img.multiply(0)).add(1)

#Relative humidity
umi_rel_terra = climatic_terra.select(['dewpoint_temperature_2m'])
umi_rel_terra = umi_rel_terra.toBands()

tdp_terra = umi_rel_terra.subtract(c3)

ea_laipelt_terra = ((img_1.exp()) \
.pow((tdp_terra.multiply(17.27)).divide((tdp_terra.add(237.3))))).multiply(0.6108)

eo_Th_hr_terra = ((img_1.exp()) \
.pow((T_air_band_C_terra.multiply(17.27)).divide((T_air_band_C_terra.add(237.3))))).multiply(0.6108)

rh_terra = ea_laipelt_terra.divide(eo_Th_hr_terra).multiply(100)

RH_terra = rh_terra.divide(100)

#wind speed
vel_vent_terra = climatic_terra.select(['u_component_of_wind_10m'])
vel_ventbands_terra = vel_vent_terra.toBands()

vel_vent_v_terra = climatic_terra.select(['v_component_of_wind_10m'])
vel_vent_vbands_terra = vel_vent_v_terra.toBands()

vel_ventbands_abs_terra = ((vel_ventbands_terra.pow(2)).add(vel_vent_vbands_terra.pow(2))).sqrt()

#Atmospheric pressure
P_atm_hr_terra = (((img_1.multiply(T_air_band_K_terra)).subtract(elevation_1.multiply(0.0065)) \
.divide(T_air_band_K_terra)).pow(5.26)).multiply(101.3)

### Loading MODIS repository
 Data base: 
* Earth's surface temperature LST - MOD11A1, MYD11A1
* Earth's surface emissivity - MOD11A1
* Zenith angle - MOD09GA
* NADIR reflectance - MCD43A4

#### Dataset MODIS

In [ ]:
# LST  (scale factor 0.02)
#TERRA 
TERRA = ee.ImageCollection('MODIS/006/MOD11A1') \
  .filterDate(start, end) \
  .select(['LST_Day_1km','Emis_31','Emis_32']) \
  .map(clipper)

# #AQUA
# LST1 = ee.ImageCollection("MODIS/006/MYD11A1") \
# .filterDate(start, end) \
#   .select(['LST_Day_1km','Emis_31','Emis_32']) \
#   .map(clipper)

# AngZ (scale factor 0.01)
ANZ = ee.ImageCollection('MODIS/006/MOD09GA') \
  .filterDate(start, end) \
  .select(['SolarZenith']) \
  .map(clipper)

# NADIR reflectance
dataset0 = ee.ImageCollection("MODIS/006/MCD43A4") \
.filterDate(start, end) \
.map(clipper)

#### Function for filling in missing values

In [ ]:
#fill in the missing values in the series using the 1 month window 
#TERRA
def replaced0 (image):
    currentDate = ee.Date(image.get('system:time_start'));
    meanImage = TERRA.filterDate(currentDate.advance(-1, 'month'), currentDate.advance(1, 'month')).mean();
    return meanImage.where(image, image).set('Date', currentDate,'system:time_start',currentDate.millis())

TERRA = TERRA.map(replaced0)

LST = TERRA.select('LST_Day_1km')
EMI1 = TERRA.select('Emis_31')
EMI2 = TERRA.select('Emis_32')

# AngZ
def replaced4 (image):
    currentDate = ee.Date(image.get('system:time_start'));
    meanImage = ANZ.filterDate(currentDate.advance(-1, 'month'), currentDate.advance(1, 'month')).mean();
    return meanImage.where(image, image).set('Date', currentDate,'system:time_start',currentDate.millis())
ANZ = ANZ.map(replaced4)

#### Scale fator

In [ ]:
#scale factor  0.001
def scale1(scene):
    # get a string representation of the date.
    dateString = ee.Date (scene.get('system:time_start')).format('yyyy-MM-dd')
    scale1 = scene.multiply(0.001)
    return scale1.rename(dateString)

#scale factor  0.02
def scale2(scene):
    # get a string representation of the date.
    dateString = ee.Date (scene.get('system:time_start')).format('yyyy-MM-dd')
    scale2 = scene.multiply(0.02)
    return scale2.rename(dateString)

#scale factor  0.002
def scale3(scene):
    # get a string representation of the date.
    dateString = ee.Date (scene.get('system:time_start')).format('yyyy-MM-dd')
    scale3 = scene.multiply(0.002)
    return scale3.rename(dateString)

#scale factor  0.01
def scale4(scene):
    # get a string representation of the date.
    dateString = ee.Date (scene.get('system:time_start')).format('yyyy-MM-dd')
    scale4 = scene.multiply(0.01)
    return scale4.rename(dateString)

In [ ]:
LST_ESC = LST.map(scale2)
# LST_ESC1 = LST1.map(scale2)

EMI1_ESC = EMI1.map(scale3)
EMI2_ESC = EMI2.map(scale3)

ANZ_ESC = ANZ.map(scale4)

In [ ]:
LST_ESCbands0 = LST_ESC.toBands().multiply(img_const_1) #TERRA
# LST_ESCbands1 = LST_ESC1.toBands().multiply(img_const_1) #AQUA

EMI1_ESCbands = EMI1_ESC.toBands().multiply(img_const_1)
EMI2_ESCbands = EMI2_ESC.toBands().multiply(img_const_1)

ANZ_ESCbands = ANZ_ESC.toBands().multiply(img_const_1)
ANZ_ESCbands = ANZ_ESCbands.selfMask()
ANZ_ESCbands = ANZ_ESCbands.reduceRegion(**{
  'reducer': ee.Reducer.median(),
  'geometry': ROI,
#   'scale': 2000,
  'bestEffort': True
  #maxPixels: 1e9
})
ANZ_ESCbands = ANZ_ESCbands.toImage()
ANZ_ESCbands = ANZ_ESCbands.clip(ROI)

### Compute albedo

In [ ]:
# Calculating albedo

# product MCD43A4
b1 = dataset0.select(['Nadir_Reflectance_Band1'])
b1 = b1.toBands().multiply(img_const_1)
b1 = b1.multiply(0.0001)
b2 = dataset0.select(['Nadir_Reflectance_Band2'])
b2 = b2.toBands().multiply(img_const_1)
b2 = b2.multiply(0.0001)
b3 = dataset0.select(['Nadir_Reflectance_Band3'])
b3 = b3.toBands().multiply(img_const_1)
b3 = b3.multiply(0.0001)
b4 = dataset0.select(['Nadir_Reflectance_Band4'])
b4 = b4.toBands().multiply(img_const_1)
b4 = b4.multiply(0.0001)
b5 = dataset0.select(['Nadir_Reflectance_Band5'])
b5 = b5.toBands().multiply(img_const_1)
b5 = b5.multiply(0.0001)
b6 = dataset0.select(['Nadir_Reflectance_Band6'])
b6 = b6.toBands().multiply(img_const_1)
b6 = b6.multiply(0.0001)
b7 = dataset0.select(['Nadir_Reflectance_Band7'])
b7 = b7.toBands().multiply(img_const_1)
b7 = b7.multiply(0.0001)

# according to the recommendations of Trezza et al. (2013)
ALB_ESCbands = (b1.multiply(0.215)).add(b2.multiply(0.266)) \
.add(b3.multiply(0.242)).add(b4.multiply(0.129)) \
.add(b5.multiply(0.00)).add(b6.multiply(0.112)) \
.add(b7.multiply(0.036))

### LST

In [ ]:
#TERRA
LST_ESCbands_terra = LST_ESCbands0
LST_ESCbands_terraC = LST_ESCbands_terra.subtract(c3)

#AQUA
# LST_ESCbands_aqua = LST_ESCbands1
# LST_ESCbands_aquaC = LST_ESCbands_aqua.subtract(c3)

#BLEND AQUA TERRA
# LST_ESCbands = LST_ESCbands0.blend(LST_ESCbands1)

### Compute Vegetation index

In [ ]:
# NDVI
NDVIbands = (b2.subtract(b1)).divide(b2.add(b1))

In [ ]:
# SAVI Miranda et al., 2020
savi = ((b2.subtract(b1)).divide(((b2.add(b1)).add(0.37)))).multiply(1.37)

In [ ]:
# LAI Miranda et al., 2020
LAI_ = (((savi.pow(2)).multiply(11)).add(0.2))

#LAI Paul et al 2014
# LAI_ = (NDVIbands.pow(3.616)).multiply(8.768)

In [ ]:
# PAI Miranda et al., 2020
PAI = (((b2.subtract((b1.sqrt()))).multiply(10.1)).add(3.1))

### DR and DOY

In [ ]:
#dr - Relative Distance Solar
def addDate(image):
    # calculate the doy band
    doy = image.date().getRelative('day', 'year')
    doyBand = ee.Image.constant(doy).uint16().rename('doy')
    doyBand = doyBand.updateMask(image.select('Nadir_Reflectance_Band2').mask())
    # calculate the dr using: dr = 1 + 0.033*cos(J*2*pi/365)
    temp = doyBand.multiply(2).multiply(math.pi).divide(365)
    temp2 = temp.cos()
    dr = temp2.multiply(0.033).add(1).rename('dr')
    # add the bands to the image
    return image.addBands([doyBand, dr]);

withDate = dataset0.map(addDate)

DR_ = (withDate.select('dr'))

doyBand = (withDate.select('doy'))

DR_bands = DR_.toBands().multiply(img_const_1)

DR_bands = DR_bands.reduceRegion(
  reducer = ee.Reducer.max(),
  geometry = ROI,
#   scale = 500,
  bestEffort = True,
)

DR_bands = DR_bands.toImage()
DR_bands = DR_bands.clip(ROI)

doyBandbands = doyBand.toBands().multiply(img_const_1)
doyBandbands = doyBandbands.add(1)

doyBandbands = doyBandbands.reduceRegion(
  reducer = ee.Reducer.max(),
  geometry = ROI,
#   scale = 500,
  bestEffort = True,
)

doyBandbands = doyBandbands.toImage()
doyBandbands = doyBandbands.clip(ROI)

## Net radiation Satellite

### Emissivity in each pixel

In [ ]:
#Emissivity in each pixel (DI LONG et al., 2010; LIANG, 2004)
band31 = EMI1_ESCbands.add(0.490)
band32 = EMI2_ESCbands.add(0.490)

EMI_MEAN = img_1.multiply(0.273).add(band31.multiply(1.778)).subtract(band31.multiply(band32).multiply(1.807))\
.subtract(band32.multiply(1.037)).add((band32.pow(2)).multiply(1.774))

EMI_MEAN = EMI_MEAN.reduceRegion(**{
  'reducer': ee.Reducer.median(),
  'geometry': ROI,
  #scale: 30,
  'bestEffort': True
  #maxPixels: 1e9
})

EMI_MEAN = EMI_MEAN.toImage()
EMI_MEAN = EMI_MEAN.clip(ROI)

### Long wave radiation emitted by the surface

In [ ]:
REMI_terra = ((LST_ESCbands_terra.pow(4)).multiply(EMI_MEAN)).multiply(sigma_SB)

### Atmospheric Transmissivity

In [ ]:
ANZ_ESCbands_rad = ANZ_ESCbands.multiply(3.1415).divide(180)
cosZ = ANZ_ESCbands_rad.cos()

Kt = img_1 #Kt = 1, for clear, clear skies

W_prec_terra = (ea_laipelt_terra.multiply(0.14).multiply(P_atm_hr_terra)).add(2.10)

temp_TSW_1_terra = ((W_prec_terra.divide(cosZ)).pow(0.4)).multiply(0.075) #aqui cosZ do terra

temp_TSW_2_terra = (P_atm_hr_terra.multiply(-0.00146)).divide(Kt.multiply(cosZ))

temp_TSW_3_terra = temp_TSW_2_terra.subtract(temp_TSW_1_terra)

TSW_1_terra = (((img_1.exp()).pow(temp_TSW_3_terra)).multiply(0.627)).add(0.35)

### Atmospheric emissivity - by duarte et al 2006

In [ ]:
#By duarte et al. 2006

EA_terra = (((ea_laipelt_terra.divide(T_air_band_K_terra)).pow(0.131)).multiply(1.545))

### Incident longwave radiation (emitted by the atmosphere)

In [ ]:
RLATM_terra = ((EA_terra.multiply(sigma_SB)).multiply(T_air_band_K_terra.pow(4)))

### Incident short wave radiation

In [ ]:
RSIN_terra = (ANZ_ESCbands_rad.cos()).multiply(1367).multiply(TSW_1_terra).multiply(img_1.divide(DR_bands))

### Net radiation Rn

In [ ]:
Rns_terra = (img_1.subtract(ALB_ESCbands).multiply(RSIN_terra))

temp_rol_atm_terra = (img_1.subtract(EMI_MEAN)).multiply(RLATM_terra)

Rn_1_terra = (Rns_terra.add(RLATM_terra.subtract(REMI_terra))).subtract(temp_rol_atm_terra)  

### Soil heat flux G

In [ ]:
G1_terra = Rn_1_terra.expression(
    '((ndvi>=0)*(((Ts*(0.0038+0.0074*albesup))*(1-0.98*ndvi**4))*Rn))+((ndvi<0)*(0.5*Rn))', {
    'Ts': LST_ESCbands_terraC,
    'ndvi':NDVIbands,
    'albesup': ALB_ESCbands,
    'Rn': Rn_1_terra
})

## roughness length for momentum  Z0M

### height of vegetation | satellite product

In [ ]:
var_name = ['GEDI_SAM_v27']
gediSAM_v27 = ee.ImageCollection('users/potapovpeter/GEDI_V27').filter(ee.Filter.inList('system:index',var_name))
gedi_band = gediSAM_v27.toBands()

roi_forest = pts.buffer(10000)
roi_forest = roi_forest.bounds()

forestCanopyHeight1 = (gedi_band.selfMask())
forestCanopyHeight1 = forestCanopyHeight1.reduceRegion(**{
  'reducer': ee.Reducer.mean(),
  'geometry': roi_forest,
  #scale: 30,
  'bestEffort': True
  #maxPixels: 1e9
})
forestCanopyHeight1 = forestCanopyHeight1.get('GEDI_SAM_v27_b1')

HGHT = img_1.multiply(ee.Number(forestCanopyHeight1))

#### Z0M by Verhoef

In [ ]:
#Constant List
PSICORR = img_1.multiply(0.2)
CD = img_1.multiply(0.01)
CR = img_1.multiply(0.35)
CD1 = img_1.multiply(20.6)

FRONTALA = PAI.divide(2)

GAM1 = (((CR.multiply(FRONTALA)).add(CD)).pow(-0.5))

GAM_m = (GAM1.lt(3.33))
GAM_m_ = GAM_m.multiply(3.33)

GAM_maior = (GAM1.gte(3.33))
GAM_maior_ = GAM_maior.multiply(GAM1)

GAM = GAM_maior_.add(GAM_m_)

DISP1 = (HGHT.multiply(
((img_1.subtract((img_1.divide(((CD1.multiply(PAI)).pow(0.5)))))) \
.add((((img_1.exp()).pow((((CD1.multiply(PAI)).pow(0.5)).multiply(-1)))) \
.divide(((CD1.multiply(PAI)).pow(0.5))))))))

PAI_menor = PAI.gt(0)

DISP_menor = PAI_menor.multiply(1)

DISP = DISP_menor.multiply(DISP1)

zom_px_px = ((HGHT.subtract(DISP)) \
.multiply(((img_1.exp()).pow((((GAM.multiply(von)).multiply(-1)).add(PSICORR))))))

### Friction velocity

#### Reference heights for climatic products

In [ ]:
zu = img_1.multiply(10)
ztair = img_1.multiply(2)

#### u* by Paul

In [ ]:
# #z0m verhoeff 1997 and paul et al. 2014
u_ast_ini_terra = (vel_ventbands_abs_terra.multiply(von))\
.divide(((zu.subtract(DISP)).divide(zom_px_px)).log())

### aerodynamic resistance

#### aerodynamic resistance by Su, Gokmen and Paul

In [ ]:
#soil moisture remote sensing
gldas_SoilMoi0_10cm_inst = ee.ImageCollection('NASA/GLDAS/V021/NOAH/G025/T3H') \
      .filter(ee.Filter.date(start, end)) \
      .select(['SoilMoi0_10cm_inst']) \
      .map(clipper)

# Daily dataset
def func_cac_mean_gldas(day):
    day = ee.Date(day)
    daily_collection = gldas_SoilMoi0_10cm_inst.filter(ee.Filter.date(day, day.advance(1, 'day')))
    convert_Rs = daily_collection.select(['SoilMoi0_10cm_inst']);
    convert_Rl_day = convert_Rs.mean().multiply(0.01);
    
    return convert_Rl_day.set('system:index', day.format('YYYY-MM-dd')) \
    .set('date', day.format('YYYY-MM-dd')) .set('system:time_start', day.millis()) \
    .set('system:time_end', day.millis())

#soil moisture day
soil_moisture_day = ee.ImageCollection.fromImages(listdates.map(func_cac_mean_gldas))
soil_moisture_day_bands = (soil_moisture_day).toBands()
soil_moisture_day_max = soil_moisture_day.max()
soil_moisture_day_min = soil_moisture_day.min()

soil_moisture_day_rel = (soil_moisture_day_bands.subtract(soil_moisture_day_min)) \
.divide(soil_moisture_day_max.subtract(soil_moisture_day_min))

#Gokmen et al 2012
a_sf = img_1.multiply(0.3)
b_sf = img_1.multiply(2.5)
c_sf = img_1.multiply(4)
temp_sf1 = img_1.add((img_1.exp()).pow((b_sf.subtract((c_sf.multiply(soil_moisture_day_rel))))))
temp_sf2 = img_1.divide(temp_sf1)
SF = a_sf.add(temp_sf2)

In [ ]:
#Constant List
pr = img_1.multiply(0.71)
visc = img_1.multiply(0.00001461)
c1 = img_1.multiply(0.320)
c2 = img_1.multiply(0.264)
cd = img_1.multiply(0.2)
ct = img_1.multiply(0.01)

Re_ast_terra = (u_ast_ini_terra.multiply(0.009)).divide(visc)
ct_ast_terra = (pr.pow(-0.667)).multiply(Re_ast_terra.pow(-0.5))


beta_terra = c1.subtract((((cd.multiply(-15.1).multiply(PAI)).exp()).multiply(c2)))

kb_1m_terra = (beta_terra.multiply(von).multiply(zom_px_px.divide(HGHT))).divide(ct_ast_terra)

# beta2_terra = ((u_ast_ini_terra.pow(2)).multiply(2)).divide(vel_ventbands_abs_terra.pow(2))

nec_terra  = (cd.multiply(PAI)).divide(beta_terra.pow(2).multiply(2))

kb_1c_terra = (cd.multiply(von))\
.divide(ct.multiply(beta_terra).multiply((img_1.subtract(((img_1.exp()).pow((nec_terra.multiply(-1))\
                                                                           .divide(2)))))).multiply(4))

kb_1s_terra = ((Re_ast_terra.pow(0.25)).multiply(2.46)).subtract(2)

ndvi_max = NDVIbands.reduceRegion(**{
  'reducer': ee.Reducer.max(),
  'geometry': ROI,
  #scale: 30,
  'bestEffort': True
  #maxPixels: 1e9
})
ndvi_max = ndvi_max.toImage()
ndvi_max = ndvi_max.clip(ROI)

ndvi_min = NDVIbands.reduceRegion(**{
  'reducer': ee.Reducer.min(),
  'geometry': ROI,
  #scale: 30,
  'bestEffort': True
  #maxPixels: 1e9
})
ndvi_min = ndvi_min.toImage()
ndvi_min = ndvi_min.clip(ROI)

fc = img_1.subtract((((NDVIbands.subtract(ndvi_max)).divide(ndvi_min.subtract(ndvi_max))).pow(0.4631)))
fs = img_1.subtract(fc)

kb_1seb_terra = ((kb_1c_terra.multiply(fc.pow(2))).add((kb_1m_terra.multiply(fc.pow(2)).multiply(fs.pow(2))))\
.add(((fs.pow(2)).multiply(kb_1s_terra)))).multiply(SF)

zoh_terra = zom_px_px.divide(((img_1.exp()).pow(kb_1seb_terra)))

#PAUL et al 2013 eq12
temp_kb_1_terra = ((zom_px_px.divide(zoh_terra)).log())

temp_rah1_terra = img_1.divide((u_ast_ini_terra.multiply(von)))

temp_rah2 = ((((zu.subtract(DISP)).divide(zom_px_px)).log()).subtract(0))

temp_rah3_terra = (temp_rah1_terra.multiply(temp_kb_1_terra))

rah_ini_terra = ((temp_rah1_terra.multiply(temp_rah2)).add(temp_rah3_terra))

## Endmembers Pixels

### Endmembers Pixels Dry/Hot

In [ ]:
# pixel hot
cand_alb0inf_terra = ALB_ESCbands.reduceRegion(**{
    'reducer': ee.Reducer.percentile([50]),
    'geometry': ROI,
    #scale: 30,
    'bestEffort': True
    #maxPixels: 1e9
  })
cand_alb0inf_terra = cand_alb0inf_terra.toImage()
cand_alb0inf_terra = cand_alb0inf_terra.clip(ROI)
cand_alb0inf_terra = cand_alb0inf_terra.multiply(img_1)

cand_alb0sup_terra = ALB_ESCbands.reduceRegion(**{
    'reducer': ee.Reducer.percentile([75]),
    'geometry': ROI,
    #scale: 30,
    'bestEffort': True
    #maxPixels: 1e9
  })
cand_alb0sup_terra = cand_alb0sup_terra.toImage()
cand_alb0sup_terra = cand_alb0sup_terra.clip(ROI).multiply(img_1)

cand_ndvi0sup_terra = NDVIbands.reduceRegion(**{
    'reducer': ee.Reducer.percentile([15]),
    'geometry': ROI,
    #scale: 30,
    'bestEffort': True
    #maxPixels: 1e9
  })
cand_ndvi0sup_terra = cand_ndvi0sup_terra.toImage()
cand_ndvi0sup_terra = cand_ndvi0sup_terra.clip(ROI).multiply(img_1)


cand_pq_alb_ndvi = (ALB_ESCbands.gt(cand_alb0inf_terra).And(ALB_ESCbands.lt(cand_alb0sup_terra))) \
.And((NDVIbands.gt(0.10).And(NDVIbands.lt(cand_ndvi0sup_terra))))

temppq0_terra = (LST_ESCbands_terra).multiply(cand_pq_alb_ndvi)

temppq_terra = temppq0_terra.selfMask()

cand_pq0inf_terra = temppq_terra.reduceRegion(**{
    'reducer': ee.Reducer.percentile([85]),
    'geometry': ROI,
    #scale: 30,
    'bestEffort': True
    #maxPixels: 1e9
  })
cand_pq0inf_terra = cand_pq0inf_terra.toImage()
cand_pq0inf_terra = cand_pq0inf_terra.clip(ROI).multiply(img_1)

cand_pq0sup_terra = temppq_terra.reduceRegion(**{
    'reducer': ee.Reducer.percentile([97]),
    'geometry': ROI,
    #scale: 30,
    'bestEffort': True
    #maxPixels: 1e9
  })
cand_pq0sup_terra = cand_pq0sup_terra.toImage()
cand_pq0sup_terra = cand_pq0sup_terra.clip(ROI).multiply(img_1)

lst_sup_inf_terra = (LST_ESCbands_terra.gt(cand_pq0inf_terra).And(LST_ESCbands_terra.lt(cand_pq0sup_terra)))

cand_pq_lst_terra = ((LST_ESCbands_terra.gte(cand_pq0inf_terra).And(LST_ESCbands_terra.lte(cand_pq0sup_terra))))\
.multiply(cand_pq_alb_ndvi)

cand_pq_terra = (cand_pq_lst_terra.selfMask()).multiply(img_1)

### Endmembers Pixels Wet/Cold

In [ ]:
# pixel cold
cand_albPF0inf_terra = ALB_ESCbands.reduceRegion(**{
    'reducer': ee.Reducer.percentile([25]),
    'geometry': ROI,
    #scale: 30,
    'bestEffort': True
    #maxPixels: 1e9
  })
cand_albPF0inf_terra = cand_albPF0inf_terra.toImage()
cand_albPF0inf_terra = cand_albPF0inf_terra.clip(ROI)

cand_albPF0sup_terra = ALB_ESCbands.reduceRegion(**{
    'reducer': ee.Reducer.percentile([50]),
    'geometry': ROI,
    #scale: 30,
    'bestEffort': True
    #maxPixels: 1e9
  })
cand_albPF0sup_terra = cand_albPF0sup_terra.toImage()
cand_albPF0sup_terra = cand_albPF0sup_terra.clip(ROI)

cand_ndvi0PFsup_terra = NDVIbands.reduceRegion(**{
    'reducer': ee.Reducer.percentile([97]),
    'geometry': ROI,
    #scale: 30,
    'bestEffort': True
    #maxPixels: 1e9
  })
cand_ndvi0PFsup_terra = cand_ndvi0PFsup_terra.toImage()
cand_ndvi0PFsup_terra = cand_ndvi0PFsup_terra.clip(ROI)


cand_pf_alb_ndvi = (ALB_ESCbands.gt(cand_albPF0inf_terra).And(ALB_ESCbands.lt(cand_albPF0sup_terra))) \
.And((NDVIbands.gt(cand_ndvi0PFsup_terra).And(NDVIbands.lt(0.99))))

temppf0_terra = (LST_ESCbands_terra).multiply(cand_pf_alb_ndvi)

temppf_terra = temppf0_terra.selfMask()

cand_pf0inf_terra = temppf_terra.reduceRegion(**{
    'reducer': ee.Reducer.percentile([0]),
    'geometry': ROI,
    #scale: 30,
    'bestEffort': True
    #maxPixels: 1e9
  })
cand_pf0inf_terra = cand_pf0inf_terra.toImage()
cand_pf0inf_terra = cand_pf0inf_terra.clip(ROI)

cand_pf0sup_terra = temppf_terra.reduceRegion(**{
    'reducer': ee.Reducer.percentile([20]),
    'geometry': ROI,
    #scale: 30,
    'bestEffort': True
    #maxPixels: 1e9
  })
cand_pf0sup_terra = cand_pf0sup_terra.toImage()
cand_pf0sup_terra = cand_pf0sup_terra.clip(ROI)

cand_pf_lst_terra = ((LST_ESCbands_terra.gte(cand_pf0inf_terra).And(LST_ESCbands_terra.lte(cand_pf0sup_terra))))\
.multiply(cand_pf_alb_ndvi)

cand_pf_terra = cand_pf_lst_terra.selfMask()

## Start loop | iterations

In [ ]:
Rn_pq_terra = (cand_pq_terra).multiply(Rn_1_terra)
Rn_pq_terra = Rn_pq_terra.reduceRegion(**{
    'reducer': ee.Reducer.median(),
    'geometry': ROI,
    #scale: 30,
    'bestEffort': True
    #maxPixels: 1e9
})

Rn_pq_terra = Rn_pq_terra.toImage()
Rn_1_quentelst_img_terra = Rn_pq_terra.clip(ROI)

Rn_pf_terra = (cand_pf_terra).multiply(Rn_1_terra)
Rn_pf_terra = Rn_pf_terra.reduceRegion(**{
    'reducer': ee.Reducer.median(),
    'geometry': ROI,
    #scale: 30,
    'bestEffort': True
    #maxPixels: 1e9
})

Rn_pf_terra = Rn_pf_terra.toImage()
Rn_friolst_img_terra = Rn_pf_terra.clip(ROI)

In [ ]:
G_pq_terra = (cand_pq_terra).multiply(G1_terra)
G_pq_terra = G_pq_terra.reduceRegion(**{
    'reducer': ee.Reducer.median(),
    'geometry': ROI,
    #scale: 30,
    'bestEffort': True
    #maxPixels: 1e9
})

G_pq_terra = G_pq_terra.toImage()
G_quentelst_img_terra = G_pq_terra.clip(ROI)

G_pf_terra = (cand_pf_terra).multiply(G1_terra)
G_pf_terra = G_pf_terra.reduceRegion(**{
    'reducer': ee.Reducer.median(),
    'geometry': ROI,
    #scale: 30,
    'bestEffort': True
    #maxPixels: 1e9
})

G_pf_terra = G_pf_terra.toImage()
G_friolst_img_terra = G_pf_terra.clip(ROI)

In [ ]:
T_pq_terra = (cand_pq_terra).multiply(LST_ESCbands_terra)
T_pq_terra = T_pq_terra.reduceRegion(**{
    'reducer': ee.Reducer.median(),
    'geometry': ROI,
    #scale: 30,
    'bestEffort': True
    #maxPixels: 1e9
})

T_pq_terra = T_pq_terra.toImage()
T_quentelst_img_terra = T_pq_terra.clip(ROI)

T_pf_terra = (cand_pf_terra).multiply(LST_ESCbands_terra)
T_pf_terra = T_pf_terra.reduceRegion(**{
    'reducer': ee.Reducer.median(),
    'geometry': ROI,
    #scale: 30,
    'bestEffort': True
    #maxPixels: 1e9
})

T_pf_terra = T_pf_terra.toImage()
T_friolst_img_terra = T_pf_terra.clip(ROI)

In [ ]:
#Slope of the vapor pressure curve at saturation
delta_p_vapor_hr_terra = (eo_Th_hr_terra.multiply(4098)).divide((((T_air_band_C_terra).add(237.3)).pow(2)))
        
const_psi_hr_terra = P_atm_hr_terra.multiply(0.000665)

densi_ar_terra = (P_atm_hr_terra.multiply(1000)).divide(T_air_band_K_terra.multiply(1.01).multiply(287.04))
        
temp_LEc_terra = delta_p_vapor_hr_terra.divide(const_psi_hr_terra.add(delta_p_vapor_hr_terra))

In [ ]:
teste_rah = ee.Number(0)
contador_loop =0
while (contador_loop<2):
       
        rah_ini_pq_terra = (cand_pq_terra).multiply(rah_ini_terra)
        rah_ini_pq_terra = rah_ini_pq_terra.reduceRegion(**{
            'reducer': ee.Reducer.median(),
            'geometry': ROI,
            #scale: 30,
            'bestEffort': True
            #maxPixels: 1e9
        })
        rah_ini_pq_terra = rah_ini_pq_terra.toImage()
        rah_ini_quentelst_img_terra = rah_ini_pq_terra.clip(ROI)
        

        rah_ini_pf_terra = (cand_pf_terra).multiply(rah_ini_terra)
        rah_ini_pf_terra = rah_ini_pf_terra.reduceRegion(**{
            'reducer': ee.Reducer.median(),
            'geometry': ROI,
            #scale: 30,
            'bestEffort': True
            #maxPixels: 1e9
        })
        rah_ini_pf_terra = rah_ini_pf_terra.toImage()
        rah_friolst_img_terra = rah_ini_pf_terra.clip(ROI)

        
        LEc_terra = (img_1.multiply(0.55)).multiply(fc).multiply(Rn_1_quentelst_img_terra.subtract(G_quentelst_img_terra)).multiply(temp_LEc_terra)

        LEc_terra_pf = (img_1.multiply(1.75)).multiply(fc).multiply(Rn_friolst_img_terra.subtract(G_friolst_img_terra)).multiply(temp_LEc_terra)

        H_pf_terra = (Rn_friolst_img_terra.subtract(G_friolst_img_terra)).subtract(LEc_terra_pf)

        dt_pf_terra = (H_pf_terra.multiply(rah_friolst_img_terra)).divide((img_1.multiply(densi_ar_terra)).multiply(1004))

        H_pq_terra = (Rn_1_quentelst_img_terra.subtract(G_quentelst_img_terra)).subtract(LEc_terra)

        dt_pq_terra = (H_pq_terra.multiply(rah_ini_quentelst_img_terra))\
        .divide((img_1.multiply(densi_ar_terra)).multiply(1004))
        
        
        b_ini_terra = (dt_pq_terra.subtract(dt_pf_terra)).divide(T_quentelst_img_terra.subtract(T_friolst_img_terra))
        a_ini_terra = dt_pf_terra.subtract(b_ini_terra.multiply(T_friolst_img_terra))
        dT_ini_terra = (LST_ESCbands_terra.multiply(b_ini_terra)).add(a_ini_terra)
        
        H_ini_terra = (dT_ini_terra.divide(rah_ini_terra)).multiply(((img_1.multiply(densi_ar_terra)).multiply(1004)))
        
        #Monin-Obukhov L
        
        L_MB_terra = H_ini_terra.expression(
            '-1.15*1004*ufp**3*Tsup/(0.41*9.81*H)', {
                'H':H_ini_terra,
                'Tsup': LST_ESCbands_terra,
                'ufp': u_ast_ini_terra
            })
        
        psi_m200_terra = H_ini_terra.expression(
            '(L<0)*((2*log((1+((1-16*(10-DISP)/L)**0.25))/2)+log((1+((1-16*(10-DISP)/L)**0.25)**2)/2)-2*atan((1-16*(10-DISP)/L)**0.25)+0.5*3.14159265359))+((L>0)*(-5*((10-DISP)/L)))', {
                'L':L_MB_terra,
                'DISP':DISP
            })

        
        psi_m2_terra = H_ini_terra.expression(
            '(L<0)*(2*log((1+((1-16*(10-DISP)/L)**0.25)**2)/2))+((L>0)*(-5*((10-DISP)/L)))', {
                'L':L_MB_terra,
                'DISP':DISP
          })
        
        u_ast_corr_terra = H_ini_terra.expression(
            '(k*u200)/(log((10-DISP)/z0)-psi_m200)', {
                'u200': u_ast_ini_terra,
                'DISP':DISP,
                'z0': zom_px_px,
                'k': 0.41,
                'psi_m200':psi_m200_terra
            })
        
        temp_rah1_corr_terra = img_1.divide((u_ast_corr_terra.multiply(von)))
        temp_rah2_corr_terra = ((((zu.subtract(DISP)).divide(zom_px_px)).log()).subtract(psi_m2_terra))
        temp_rah3_corr_terra = (temp_rah1_corr_terra.multiply(temp_kb_1_terra))

        rah_corr_terra = ((temp_rah1_corr_terra.multiply(temp_rah2_corr_terra)).add(temp_rah3_corr_terra))
        
        #Error rah
        rah_dif_terra =  rah_ini_terra.subtract(rah_corr_terra)
        rah_dif_terra = ee.Algorithms.If(rah_dif_terra.lt(0), rah_dif_terra.multiply(-1), rah_dif_terra )
        rah_dif_terra = img_1.multiply(rah_dif_terra)
        teste_rah_terra = rah_dif_terra.multiply(100).divide(rah_corr_terra)

        rah_ini_terra = rah_corr_terra
        u_ast_ini_terra = u_ast_corr_terra
        
        contador_loop = contador_loop + 1

## Sensible heat flux H

In [ ]:
H_corr1i0_terra = ((H_ini_terra).lt(0)).multiply(0.01)
H_corr1i1_terra = ((H_ini_terra).gte(0)).multiply(H_ini_terra)

H_end_terra = H_corr1i0_terra.add((H_corr1i1_terra))

## Latent heat flux LE

In [ ]:
LE_terra = ((Rn_1_terra).subtract(G1_terra)).subtract(H_end_terra)

LE_F1_terra = ((LE_terra).lt(0)).multiply(0.01)
LE_F2_terra = ((LE_terra).gte(0)).multiply(LE_terra)

LE_F_terra = LE_F1_terra.add((LE_F2_terra))

## Compute evapotranspiration day

In [ ]:
pre_calc = ((img_1.multiply(86400)).divide(((img_1.multiply(2.501)).subtract((T_air_band_C_terra.multiply(0.00236)))))).multiply(0.000001)

temp_ETa1_terra = (LE_F_terra.divide((Rn_1_terra.subtract(G1_terra)))).multiply(pre_calc)
FE_f = temp_ETa1_terra

ETa_SEBAL_2_terra = FE_f.multiply(Rn24)